In [1]:
import importlib
import frodo
from frodo import *
import numpy as np
import secrets
import discrete_distr
from discrete_distr import *

In [5]:
importlib.reload(frodo)
from frodo import *

In [128]:
importlib.reload(discrete_distr)
from discrete_distr import *

In [ ]:
def sample_fail(ps, s, fp, t, prg):
    '''
    fp = filter pattern, e.g. fp = [12, 12, 12, 10, 10]
    sample w such that  s dot w  > t
    '''
    n = len(S)
    w = np.empty(n, dtype=np.int16)
    w[0:len(fp)] = fp
    for i in range(len(fp), n-1):
        w[i] = ps.sample(int.from_bytes(int.from_bytes(prg.bytes(2), 'little')) )
    

In [17]:
def sample_dec_error(ps, S, E, prg):
    Sp = ps.sample_array(prg, ps.m_bar, ps.n)
    Ep = ps.sample_array(prg, ps.m_bar, ps.n)
    Epp = ps.sample_array(prg, ps.m_bar, ps.n_bar)
    Eppp = np.matmul(Sp, E) + Epp - np.matmul(Ep, S)
    return Eppp
def dec_fail(ps, Eppp):
    '''return True if fail'''
    b = ps.q / (2 ** ps.B)
    lower_t = -b/2
    upper_t = b/2
    y = np.mod(Eppp, ps.q)
    mask = (y >= ps.q/2)
    y[mask] = y[mask] - ps.q
    return not np.all(np.logical_and(lower_t <= y, y < upper_t))
def empirical_dfr(ps, S, E, prg, total= 1 << 10):
    fails = 0
    for i in range(total):
        Eppp = sample_dec_error(ps, S, E, prg)
        if dec_fail(ps, Eppp):
            fails += 1
    return fails, total, 1.0*fails/total
def dec_fail_pr(ps, distr, lower_t=None, upper_t=None):
    pr = 0
    b = ps.q / (2 ** ps.B)
    if lower_t == None:
        lower_t = -b/2
    if upper_t == None:
        upper_t = b/2
    for x, p in distr.items():
        #y = min(x, ps.q - x)
        y = x if x < ps.q / 2 else x - ps.q
        if not (lower_t <= y < upper_t):
            pr += p
    return pr
def center_mod(distr, q):
    return {x if x < q/2 else x - q: p for (x,p) in distr.items()}
def per_key_honest_dfr(ps, S, E, lower_t=None, upper_t = None, columns=None):
    '''dfr for each entry in each column'''
    n, m = E.shape
    
    if columns==None:
        C = iter(range(m)) 
    else:
        try:
            C = iter(columns)
        except TypeError:
            C = [columns]
            
    Sd = [{} for c in C]
    Ed = [{} for c in C]
    for k, c in enumerate(C):
        for i in range(n):
            Sd[k][S[i,c]] = Sd[k].get(S[i, c], 0) + 1
            Ed[k][E[i,c]] = Ed[k].get(E[i, c], 0) + 1
    
        #print(center_mod(Sd[k], ps.q))
        #print(center_mod(Ed[k], ps.q))
            
    D = [{0:1} for c in C]
    for k, c in enumerate(C):
        for e,r in Ed[k].items():
            a = pdf_product(ps.chi, {e:1}, ps.q) # S'E
            A = nfoldconvolution(r, a, ps.q)
            D[k] = convolution(D[k], A, ps.q)
        for s,r in Sd[k].items():
            b = pdf_product(ps.chi, {s:1}, ps.q)
            B = nfoldconvolution(r, b, ps.q) # E'S
            D[k] = convolution(D[k], B, ps.q)
        D[k] = convolution(D[k], ps.chi)
    
    
    dfr = [dec_fail_pr(ps, d, lower_t, upper_t) for d in D]
    if len(D) == 1:
        D = D[0]
        dfr = dfr[0]
    return dfr
def test_simple_key_recovery(ps, s, prg, lower_t=None, upper_t=None, ntries=5000):
    b = ps.q / (1 << ps.B)
    if lower_t == None:
        lower_t = -b/2
    if upper_t == None:
        upper_t = b/2
    n = s.shape[0]
    r = np.zeros(n) 
    nfails = 0
    for i in range(ntries):
        w = ps.sample_array(prg, n)
        d = np.dot(s, w)
        if d >= upper_t:
            nfails += 1
            r += w
        elif d < lower_t:
            nfails += 1
            r -= w
    return r, nfails
def work_estimate(ps, t, v):
    '''
    return
    number of fail ciphertexts,
    number of dec,
    number of hashes
    '''

In [1]:
ps.chi[10]

NameError: name 'ps' is not defined

In [2]:
ps = FrodoPS(640)
seed = secrets.token_bytes(16)
prg = np.random.default_rng(list(seed))

In [3]:
ps.sigma

2.8

In [4]:
ave = 0
for x, p in ps.chi.items():
    ave += abs(x)*p

In [9]:
chip = {}
P = sum([p for x, p in ps.chi.items() if x >= 0])
for x, p in ps.chi.items():
    if x < 0:
        continue
    chip[x] = p/P

In [10]:
ave = 0
for x, p in chip.items():
    ave += x*p
print(ave)

1.9462739228055166


In [4]:
S = ps.sample_array(prg, ps.n, ps.n_bar)
E = ps.sample_array(prg, ps.n, ps.n_bar)
for i in range(ps.n_bar):
    for j in range(ps.n):
        if E[j, i] >= 0:
            E[j, i] += 256
            break

In [4]:
S = ps.sample_array(prg, ps.n, ps.n_bar)

In [16]:
E = ps.sample_array(prg, ps.n- 1, ps.n_bar)

In [17]:
for i in range(ps.n_bar):
    print( np.linalg.norm(np.concatenate((S[:, i], E[:, i]))) )

104.04806581575652
103.92304845413264
99.95498987044118
97.3755616158387
101.25709851659784
102.22524150130437
100.98514742277698
103.4456378974


In [18]:
(2*ps.n - 1)**0.5 * ps.sigma

100.13670655658692

In [24]:
r = 0
for x, p in ps.chi.items():
    r += x**2 * p
r**0.5

2.814560876884261

In [25]:
(2*ps.n - 1)**0.5 * r**0.5

100.657448790789

In [11]:
S = ps.sample_array(prg, 100, 1)
E = ps.sample_array(prg, 100, 1)

In [14]:
a = np.array([1,2,3])
b = np.array([4,5,6])
np.concatenate((a,b))

array([1, 2, 3, 4, 5, 6])

In [16]:
s

array([-4,  4,  0, -2,  0,  2,  4,  1, -4,  0, -2,  6,  1, -4,  3,  0, -1,
       -1, -4, -4, -1,  0, -2,  3, -1, -2, -3,  6,  2,  0,  2,  4,  0,  0,
       -3,  3, -2,  3,  0,  6, -4, -7,  5, -5, -1,  4,  0, -3, -3,  5,  2,
       -3, -2, -1,  1,  3,  0,  1,  0, -1,  0,  0, -1, -1,  0, -1, -3, -7,
       -4,  1,  1,  1, -1, -4,  1,  3, -1,  2,  1,  0,  3,  3, -1,  0, -7,
        2,  2,  4, -1,  0, -1,  2, -2, -1, -2, -5,  0, -1,  0, -1],
      dtype=int16)

In [18]:
r, nfails = test_simple_key_recovery(ps, s, prg, -150, 150)

In [19]:
nfails

298

In [23]:
r/nfails

array([-0.8557047 ,  0.91610738, -0.00671141, -0.43288591, -0.09731544,
        0.7114094 ,  0.81879195,  0.0704698 , -0.72147651,  0.15100671,
       -0.25503356,  1.39932886,  0.22818792, -0.91275168,  0.57718121,
       -0.11073826, -0.19463087, -0.23825503, -1.02013423, -0.98657718,
       -0.09395973,  0.18791946, -0.65436242,  0.8590604 ,  0.04362416,
       -0.47315436, -0.55369128,  1.3557047 ,  0.62751678, -0.14765101,
        0.32885906,  1.11744966,  0.01677852,  0.01677852, -0.70805369,
        0.86577181, -0.44966443,  0.52013423,  0.0704698 ,  1.41275168,
       -0.98993289, -1.6442953 ,  1.06711409, -1.24832215, -0.19127517,
        0.74832215, -0.01342282, -0.44295302, -0.91275168,  1.13422819,
        0.40939597, -0.7114094 , -0.51342282,  0.01006711,  0.41275168,
        0.76510067,  0.30536913,  0.2114094 , -0.02348993, -0.23154362,
       -0.09731544, -0.09060403, -0.03020134, -0.17785235,  0.15100671,
       -0.18791946, -0.90604027, -1.58389262, -0.53691275,  0.40

In [22]:
s

array([-4,  4,  0, -2,  0,  2,  4,  1, -4,  0, -2,  6,  1, -4,  3,  0, -1,
       -1, -4, -4, -1,  0, -2,  3, -1, -2, -3,  6,  2,  0,  2,  4,  0,  0,
       -3,  3, -2,  3,  0,  6, -4, -7,  5, -5, -1,  4,  0, -3, -3,  5,  2,
       -3, -2, -1,  1,  3,  0,  1,  0, -1,  0,  0, -1, -1,  0, -1, -3, -7,
       -4,  1,  1,  1, -1, -4,  1,  3, -1,  2,  1,  0,  3,  3, -1,  0, -7,
        2,  2,  4, -1,  0, -1,  2, -2, -1, -2, -5,  0, -1,  0, -1],
      dtype=int16)

In [6]:
for i in range(8):
    dfr =  per_key_honest_dfr(ps, S, E, i)
    print( log(dfr, 2) )

-25.857986981301714
-26.848708282179114
-26.401377340135078
-26.130083172019262
-28.074782265960827
-27.684214809289504
-27.60659824432083
-26.162024226327595


In [45]:
empirical_dfr(ps, S, E, prg, 100)

(0, 100, 0.0)

In [46]:
M = 0
for i in range(1000):
    Eppp = sample_dec_error(ps,S,E,prg)
    m = np.amax(np.absolute(Eppp))
    if m > M:
        M = m
print(M)

3272


In [3]:
a = np.empty(5, dtype=np.int16)

In [8]:
b = *a.shape
b

SyntaxError: can't use starred expression here (<ipython-input-8-9afb1e661e61>, line 4)

In [20]:
for i in range(2**20):
    x = ps.sample(int.from_bytes(prg.bytes(2),'little'))
    if x == 12:
        print(i, log(i, 2))
        break

33342 15.025053025671644


In [18]:
log(72097, 2)

16.137651608846774

In [15]:
importlib.reload(discrete_distr)
from discrete_distr import *

In [16]:
convolution({-10:1, 10:0}, {1:1}, 4)

[1.]
[1.]


{3: 1.0}

In [4]:
def dict_to_arr(x):
        """Convert a distribution into an (offset, array) pair.
        """
        min_x, max_x = min(x.keys()), max(x.keys())
        a = np.zeros(max_x - min_x + 1, dtype=np.float128)
        for i, p in x.items():
            a[i - min_x] += p
        return min_x, a

In [14]:
a = b = {}
a[0] =1
a, b

({0: 1}, {0: 1})

In [7]:
hp = {(256, None): 1}
log(Frodo_hammer_dfr(ps, hp), 2)

-27.827578047191004

In [8]:
hp = {(256, lambda x: x <= -12): 1}
log(Frodo_hammer_dfr(ps, hp, upper_t = 99999), 2)

-12.968327567495235

In [24]:
for x, y in enumerate([2,4,6]):
    print(x, y)

0 2
1 4
2 6


In [9]:
def f(a):
    a0 = a
    a = {}
    for v, p in a0.items():
        a[v] = 2*p
    return a

In [10]:
b = {1: 2}
f(b)

{1: 4}

In [11]:
b

{1: 2}

In [68]:
def exact_failure_prob_pke(noise, q, n, w, reclen):
    """Compute the probability of failure of PKE (exactly).

    Args:
      noise: The noise distribution as a dictionary.
      q: Modulus.
      n: Vector length.
      w: Number of extracted bits per coordinate.
      reclen: Number of bits to be extracted.

    Returns:
      The probability of failure.
    """

    def pr_rec_failure_pke(x):
        # Probability of failing to recover from an error of magnitude x.
        #        0% if -B/2 <= x < B/2
        #        100% if x >= B/2 or x < -B/2

        x = min(x, q - x)
        b = q / (2 ** w)
        return 0 if -b / 2 <= x < b / 2 else 1.

    noise_sqr = pdf_product(noise, noise, q)

    v = nfoldconvolution(2 * n, noise_sqr, q)

    v = convolution(v, noise, q)  # v = 2n * (noise^2) + noise

    exact_pr = {x: p * pr_rec_failure_pke(x) for (x, p) in v.items()}

    r1r2 = (reclen + w - 1) // w

    failure_pr = r1r2 * sum(exact_pr.values())  # union bound

    return failure_pr, exact_pr.keys()

In [70]:
pr, ks = exact_failure_prob_pke(ps.chi, ps.q, ps.n, 2, 16)

In [71]:
ks

dict_keys([23031, 23032, 23033, 23034, 23035, 23036, 23037, 23038, 23039, 23040, 23041, 23042, 23043, 23044, 23045, 23046, 23047, 23048, 23049, 23050, 23051, 23052, 23053, 23054, 23055, 23056, 23057, 23058, 23059, 23060, 23061, 23062, 23063, 23064, 23065, 23066, 23067, 23068, 23069, 23070, 23071, 23072, 23073, 23074, 23075, 23076, 23077, 23078, 23079, 23080, 23081, 23082, 23083, 23084, 23085, 23086, 23087, 23088, 23089, 23090, 23091, 23092, 23093, 23094, 23095, 23096, 23097, 23098, 23099, 23100, 23101, 23102, 23103, 23104, 23105, 23106, 23107, 23108, 23109, 23110, 23111, 23112, 23113, 23114, 23115, 23116, 23117, 23118, 23119, 23120, 23121, 23122, 23123, 23124, 23125, 23126, 23127, 23128, 23129, 23130, 23131, 23132, 23133, 23134, 23135, 23136, 23137, 23138, 23139, 23140, 23141, 23142, 23143, 23144, 23145, 23146, 23147, 23148, 23149, 23150, 23151, 23152, 23153, 23154, 23155, 23156, 23157, 23158, 23159, 23160, 23161, 23162, 23163, 23164, 23165, 23166, 23167, 23168, 23169, 23170, 23171, 23

In [9]:
a = np.empty((3,3), dtype=np.int16)

In [13]:
a

array([[ -9136,    599,      8],
       [     0,  13576, -32717],
       [     1,      0,     32]], dtype=int16)

In [17]:
np.mod(a, 3)

array([[2, 2, 2],
       [0, 1, 1],
       [1, 0, 2]], dtype=int16)

In [14]:
a[0:3] = [16]*3

In [16]:
a[0:] = [1,2,3]

ValueError: cannot copy sequence with size 3 to array axis with dimension 10